# Project "Battle of Neighbourhood"

##### Problem Statement : To Determine warehouse location within Etobicokeo to cater to supply of food products to the maximum food joints possible today and for future


In [458]:
#  Importing Necessary packages

In [459]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files



import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Assignment Part 1 - Data Scrapping from Wiki Page

In [460]:
# import the library we use to open URLs
import urllib.request

In [461]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [462]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [463]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [464]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [465]:
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [466]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [467]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [468]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [469]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [470]:
df["Neighbourhood"] = df["Neighbourhood"].str.replace("\n","")
df["Postal Code"] = df["Postal Code"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [471]:
r_df = df[df.Borough != 'Not assigned']
r_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [472]:
r_df = r_df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
r_df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [473]:
if "Not assigned" in r_df["Neighbourhood"]: 
            r_df["Neighbourhood"] = r_df["Borough"]
r_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [474]:
print("Shape: ", r_df.shape)

Shape:  (103, 3)


###                                                         Appending latitude and the longitude coordinates of each neighbourhood

In [475]:
filename = "https://cocl.us/Geospatial_data"
l_df = pd.read_csv(filename)
l_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [476]:
df_toronto = pd.merge(r_df, l_df, how='inner', on = 'Postal Code')
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [477]:
# Filtering only Etobicoke neighbourhood

new = df_toronto["Borough"].isin(["Etobicoke"]) 
df_etob = df_toronto[new]
df_etob.head()
  

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
89,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
91,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
92,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [537]:
df_etob

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
89,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
91,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
92,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999
93,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
94,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


##  Webscarapping Etobicoke Demographic Data

In [479]:
url2 = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"

In [480]:
# open the url using urllib.request and put the HTML into the page variable
page2 = urllib.request.urlopen(url2)


In [481]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [482]:
soup = BeautifulSoup(page2, "lxml")
soup.title.string

'Demographics of Toronto neighbourhoods - Wikipedia'

In [483]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
Demographic_tables=soup.find_all("table")
Demographic_tables

[<table class="box-Update plainlinks metadata ambox ambox-content ambox-Update" role="presentation"><tbody><tr><td class="mbox-image"><div style="width:52px"><img alt="Ambox current red Americas.svg" data-file-height="290" data-file-width="360" decoding="async" height="34" src="//upload.wikimedia.org/wikipedia/commons/thumb/5/53/Ambox_current_red_Americas.svg/42px-Ambox_current_red_Americas.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/5/53/Ambox_current_red_Americas.svg/63px-Ambox_current_red_Americas.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/5/53/Ambox_current_red_Americas.svg/84px-Ambox_current_red_Americas.svg.png 2x" width="42"/></div></td><td class="mbox-text"><div class="mbox-text-span">This article's <b><a href="/wiki/Wikipedia:Accuracy_dispute" title="Wikipedia:Accuracy dispute">factual accuracy</a> may be compromised due to out-of-date information</b>.<span class="hide-when-compact"> Please update this article to reflect recent events or n

In [484]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th width="20%">Name
</th>
<th width="5%">FM
</th>
<th width="20%">Census Tracts
</th>
<th width="5%">Population
</th>
<th width="5%">Land area (km2)
</th>
<th width="5%">Density (people/km2)
</th>
<th width="5%">% Change in Population since 2001
</th>
<th width="5%">Average Income
</th>
<th width="5%">Transit Commuting %
</th>
<th width="5%">% Renters
</th>
<th width="10%">Second most common language (after English) by name
</th>
<th width="10%">Second most common language (after English) by percentage
</th>
<th width="10%">Map
</th></tr>
<tr>
<td><b>Toronto <a class="mw-redirect" href="/wiki/Census_metropolitan_area" title="Census metropolitan area">CMA</a> Average</b>
</td>
<td>
</td>
<td>All
</td>
<td><b>5,113,149</b>
</td>
<td><b>5903.63</b>
</td>
<td><b>866</b>
</td>
<td><b>9.0</b>
</td>
<td><b>40,704</b>
</td>
<td><b>10.6</b>
</td>
<td><b>11.4</b>
</td>
<td>
</td>
<td>
</td>
<td>
</td></tr>
<tr>
<td><a href="/wiki/Agincourt,_Toronto

In [485]:
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
I=[]
J=[]
K=[]
L=[]
M=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==13:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True))
        H.append(cells[7].find(text=True))
        I.append(cells[8].find(text=True))
        J.append(cells[9].find(text=True))
        K.append(cells[10].find(text=True))
        L.append(cells[10].find(text=True))
        M.append(cells[10].find(text=True))
        

In [486]:
df=pd.DataFrame(A,columns=['A'])
df['B']=B
df['C']=C
df['D']=D
df['E']=E
df['F']=F
df['G']=G
df['H']=H
df['I']=I
df['J']=J
df['K']=K
df['L']=L
df['M']=M
df

,A,B,C,D,E,F,G,H,I,J,K,L,M
0,Toronto,\n,All\n,"5,113,149",5903.63,866,9.0,"40,704",10.6,11.4,\n,\n,\n
1,Agincourt,S\n,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...","44,577\n",12.45\n,3580\n,4.6\n,"25,750\n",11.1\n,5.9\n,Cantonese (19.3%)\n,Cantonese (19.3%)\n,Cantonese (19.3%)\n
2,Alderwood,E\n,"0211.00, 0212.00\n","11,656\n",4.94\n,2360\n,-4.0\n,"35,239\n",8.8\n,8.5\n,Polish (6.2%)\n,Polish (6.2%)\n,Polish (6.2%)\n
3,Alexandra Park,OCoT\n,0039.00\n,"4,355\n",0.32\n,"13,609\n",0.0\n,"19,687\n",13.8\n,28.0\n,Cantonese (17.9%)\n,Cantonese (17.9%)\n,Cantonese (17.9%)\n
4,Allenby,OCoT\n,0140.00\n,"2,513\n",0.58\n,4333\n,-1.0\n,"245,592\n",5.2\n,3.4\n,Russian (1.4%)\n,Russian (1.4%)\n,Russian (1.4%)\n
5,Amesbury,NY\n,"0280.00, 0281.01, 0281.02\n","17,318\n",3.51\n,"4,934\n",1.1\n,"27,546\n",16.4\n,19.7\n,Spanish (6.1%)\n,Spanish (6.1%)\n,Spanish (6.1%)\n
6,Armour Heights,NY\n,0298.00\n,"4,384\n",2.29\n,1914\n,2.0\n,"116,651\n",10.8\n,16.1\n,Russian (9.4%)\n,Russian (9.4%)\n,Russian (9.4%)\n
7,Banbury,NY\n,0267.00\n,"6,641\n",2.72\n,2442\n,5.0\n,"92,319\n",6.1\n,4.8\n,Unspecified Chinese (5.1%)\n,Unspecified Chinese (5.1%)\n,Unspecified Chinese (5.1%)\n
8,Bathurst Manor,NY\n,"0297.01, 0310.01, 0310.02\n","14,945\n",4.69\n,3187\n,12.3\n,"34,169\n",13.4\n,18.6\n,Russian (9.5%)\n,Russian (9.5%)\n,Russian (9.5%)\n
9,Bay Street Corridor,OCoT\n,0062.01\n,"4,787\n",0.11\n,"43,518\n",3.0\n,"40,598\n",17.1\n,49.3\n,Mandarin (9.6%)\n,Mandarin (9.6%)\n,Mandarin (9.6%)\n


In [487]:
demog_df = df.drop(["C","D","E", "I", "J", "K", "L", "M"], axis= 1)
demog_df

,A,B,F,G,H
0,Toronto,\n,866,9.0,"40,704"
1,Agincourt,S\n,3580\n,4.6\n,"25,750\n"
2,Alderwood,E\n,2360\n,-4.0\n,"35,239\n"
3,Alexandra Park,OCoT\n,"13,609\n",0.0\n,"19,687\n"
4,Allenby,OCoT\n,4333\n,-1.0\n,"245,592\n"
5,Amesbury,NY\n,"4,934\n",1.1\n,"27,546\n"
6,Armour Heights,NY\n,1914\n,2.0\n,"116,651\n"
7,Banbury,NY\n,2442\n,5.0\n,"92,319\n"
8,Bathurst Manor,NY\n,3187\n,12.3\n,"34,169\n"
9,Bay Street Corridor,OCoT\n,"43,518\n",3.0\n,"40,598\n"


In [488]:
dg_df = demog_df.rename(columns={"A": "Neighbourhood", "B": "Borough", "F": "Density(people/km2)", "G": "% change in population since 2001","H": "Average Income"})
dg_df

,Neighbourhood,Borough,Density(people/km2),% change in population since 2001,Average Income
0,Toronto,\n,866,9.0,"40,704"
1,Agincourt,S\n,3580\n,4.6\n,"25,750\n"
2,Alderwood,E\n,2360\n,-4.0\n,"35,239\n"
3,Alexandra Park,OCoT\n,"13,609\n",0.0\n,"19,687\n"
4,Allenby,OCoT\n,4333\n,-1.0\n,"245,592\n"
5,Amesbury,NY\n,"4,934\n",1.1\n,"27,546\n"
6,Armour Heights,NY\n,1914\n,2.0\n,"116,651\n"
7,Banbury,NY\n,2442\n,5.0\n,"92,319\n"
8,Bathurst Manor,NY\n,3187\n,12.3\n,"34,169\n"
9,Bay Street Corridor,OCoT\n,"43,518\n",3.0\n,"40,598\n"


In [489]:
dg_df["Borough"] = dg_df["Borough"].str.replace("\n","")
dg_df["Density(people/km2)"] = dg_df["Density(people/km2)"].str.replace("\n","")
dg_df["% change in population since 2001"] = dg_df["% change in population since 2001"].str.replace("\n","")
dg_df["Average Income"] = dg_df["Average Income"].str.replace("\n","")

dg_df = dg_df.drop([0], axis=0)
dg_df

,Neighbourhood,Borough,Density(people/km2),% change in population since 2001,Average Income
1,Agincourt,S,3580,4.6,"25,750"
2,Alderwood,E,2360,-4.0,"35,239"
3,Alexandra Park,OCoT,"13,609",0.0,"19,687"
4,Allenby,OCoT,4333,-1.0,"245,592"
5,Amesbury,NY,"4,934",1.1,"27,546"
6,Armour Heights,NY,1914,2.0,"116,651"
7,Banbury,NY,2442,5.0,"92,319"
8,Bathurst Manor,NY,3187,12.3,"34,169"
9,Bay Street Corridor,OCoT,"43,518",3.0,"40,598"
10,Bayview Village,NY,"2,966",41.6,"46,752"


In [490]:
new = dg_df["Borough"].isin(["E"]) 
Etob_Dgdata = dg_df[new].drop((["Borough"]), axis=1)
Etob_Dgdata

,Neighbourhood,Density(people/km2),% change in population since 2001,Average Income
2,Alderwood,2360,-4.0,"35,239"
24,Centennial,2544,0.5,"34,867"
28,Clairville,1268,-3.3,"26,610"
48,Eatonville,1699,4.3,"36,206"
51,Eringate,3282,-3.4,"34,789"
72,Humber Bay Shores,7588,-5.7,"39,186"
73,Humber Heights,2766,8.3,"39,738"
75,Humber Valley Village,2652,-0.1,"80,618"
78,Humberwood,421,8.0,"29,576"
81,Islington – Six Points,4106,3.7,"43,570"


### Final Etobicokeo refined demographic Data 

In [491]:
etobdmg = pd.read_csv(r"C:\Users\DEEPTI NARANG\Desktop\demograpic data.csv") 
etobdmg

,Neighbourhood,Density(people/km2),% change in population since 2001,Average Income
0,"Alderwood, Long Branch",2360,-4.0,35239
1,"Northwest, West Humber - Clairville",1268,-3.3,26610
2,"New Toronto, Mimico South, Humber Bay Shores",7588,-5.7,39186
3,"Islington Avenue, Humber Valley Village",2652,-0.1,80618
4,"Kingsview Village, St. Phillips, Martin Grove ...",4013,-6.2,32004
5,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",3507,1.0,51695
6,"West Deane Park, Princess Gardens, Martin Grov...",2249,1.0,80607
7,"Old Mill South, King's Mill Park, Sunnylea, Hu...",3366,-1.1,51398
8,"The Kingsway, Montgomery Road, Old Mill North",3403,7.9,110944
9,Westmount,5932,5.0,35183


## Explore and cluster the neighborhoods in Toronto

In [492]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-32::ca-certificates-2020.1.1-0, anaconda/win-32::certifi-2019.11.28-py37_0, anaconda/win-32::conda-4.8.3-py37_0, anaconda/win-32::openssl-1.1.1d-he774522_4
  - anaconda/win-32::ca-certificates-2020.1.1-0, anaconda/win-32::certifi-2019.11.28-py37_0, anaconda/win-32::openssl-1.1.1d-he774522_4, defaults/win-32::conda-4.8.3-py37_0
  - anaconda/win-32::ca-certificates-2020.1.1-0, anaconda/win-32::certifi-2019.11.28-py37_0, anaconda/win-32::conda-4.8.3-py37_0, defaults/win-32::openssl-1.1.1d-he774522_4
  - anaconda/win-32::ca-certificates-2020.1.1-0, anaconda/win-32::certifi-2019.11.28-py37_0, defaults/win-32::conda-4.8.3-py37_0, defaults/win-32::openssl-1.1.1d-he774522_4
  - anaconda/win-32::certifi-2019.11.28-py37_0, anaconda/win-32::conda-4.8.3-py37_0, defaults/win-32::ca-certificates-2020.1.1-0, defaults/win-32::openssl-1.1.1d-he774522_4
  - anaconda/win-32::certifi-2019.11.28-py37_0, defaults/win-32::ca-certificates-2020.1.1-0, defaul

###  Exploring Toronto

In [493]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [494]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
toronto_map

In [495]:
CLIENT_ID = 'X5ZNDHYXCMDPNX0ZBOKXNUO20N5AKSSXWCMIP50FQSDOTALI' # your Foursquare ID
CLIENT_SECRET = 'HEXCTFUJWHTTTRADD2YEMG2C02DDG4ARGUUKYN0BPDBEXYON' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X5ZNDHYXCMDPNX0ZBOKXNUO20N5AKSSXWCMIP50FQSDOTALI
CLIENT_SECRET:HEXCTFUJWHTTTRADD2YEMG2C02DDG4ARGUUKYN0BPDBEXYON


In [496]:
for lat, lng, borough, neighbourhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map


###  Exploring the neighborhood

In [497]:
def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [498]:
# Extracting venues details only for Etobicokeo

etob_venues = getNearbyVenues(names=df_etob['Neighbourhood'],
                                   latitudes=df_etob['Latitude'],
                                   longitudes=df_etob['Longitude']
                                  )
etob_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Huevos Gourmet,43.601188,-79.503717,Mexican Restaurant
1,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
2,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Sweet Olenka's,43.601099,-79.500325,Dessert Shop
3,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Cellar Door Restaurant,43.600221,-79.507638,Italian Restaurant
4,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Birds and Beans Cafe,43.613942,-79.489062,Café


In [499]:
# Battle cell added  (venues only for etob)
etob_venues.tail()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
355,"Northwest, West Humber - Clairville",43.706748,-79.594054,Delta Hotels by Marriott Toronto Airport & Con...,43.688993,-79.577907,Hotel
356,"Northwest, West Humber - Clairville",43.706748,-79.594054,Tim Hortons,43.713177,-79.580186,Coffee Shop
357,"Northwest, West Humber - Clairville",43.706748,-79.594054,Threshold Aviation,43.683147,-79.593388,Hobby Shop
358,"Northwest, West Humber - Clairville",43.706748,-79.594054,Wok of Asia,43.734152,-79.605776,Chinese Restaurant
359,"Northwest, West Humber - Clairville",43.706748,-79.594054,Sushi Ya,43.690363,-79.575964,Sushi Restaurant


In [500]:
etob_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Huevos Gourmet,43.601188,-79.503717,Mexican Restaurant
1,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
2,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Sweet Olenka's,43.601099,-79.500325,Dessert Shop
3,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Cellar Door Restaurant,43.600221,-79.507638,Italian Restaurant
4,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Birds and Beans Cafe,43.613942,-79.489062,Café
5,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,SanRemo Bakery,43.618542,-79.499485,Bakery
6,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Prince Of Wales Park,43.598797,-79.499001,Park
7,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Black Oak Brewing,43.613747,-79.516339,Brewery
8,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Colonel Samuel Smith Park,43.592141,-79.512305,Park
9,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Tich - Modern Indian Cuisine,43.617585,-79.487298,Indian Restaurant


In [501]:
# Battle cell added

print('There are {} uniques categories.'.format(len(etob_venues['Venue Category'].unique())))
list(etob_venues['Venue Category'].unique())

There are 84 uniques categories.


['Mexican Restaurant',
 'Liquor Store',
 'Dessert Shop',
 'Italian Restaurant',
 'Café',
 'Bakery',
 'Park',
 'Brewery',
 'Indian Restaurant',
 'General Entertainment',
 'Sushi Restaurant',
 'Coffee Shop',
 'Theater',
 'Restaurant',
 'Ice Cream Shop',
 'Skating Rink',
 'Pizza Place',
 'Burrito Place',
 'Japanese Restaurant',
 'Movie Theater',
 'Supermarket',
 'Middle Eastern Restaurant',
 'Thai Restaurant',
 'Breakfast Spot',
 'Grocery Store',
 'Burger Joint',
 'Seafood Restaurant',
 'Gourmet Shop',
 'Bookstore',
 'New American Restaurant',
 'Fast Food Restaurant',
 'Sandwich Place',
 'Toy / Game Store',
 'South American Restaurant',
 'Climbing Gym',
 'Greek Restaurant',
 'Department Store',
 'Discount Store',
 'Shopping Mall',
 'Sporting Goods Shop',
 'Pub',
 'Gym',
 'Pharmacy',
 'Tea Room',
 'French Restaurant',
 'Tapas Restaurant',
 'Pool Hall',
 'Gastropub',
 'Indie Movie Theater',
 'Golf Course',
 'Taco Place',
 'Yoga Studio',
 'American Restaurant',
 'Event Space',
 'Garden',
 'D

In [502]:
# Battle cell added

# one hot encoding
etob_onehot = pd.get_dummies(etob_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etob_onehot['Neighbourhood'] = etob_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [etob_onehot.columns[-1]] + list(etob_onehot.columns[:-1])
etob_onehot = etob_onehot[fixed_columns]

etob_onehot.head()



,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Pharmacy,Pizza Place,Pool Hall,Pub,Racecourse,Racetrack,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,South American Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [503]:
 # Battle cell added

etob_grouped = etob_onehot.groupby('Neighbourhood').sum().reset_index()
etob_grouped.head()

,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Coffee Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Pharmacy,Pizza Place,Pool Hall,Pub,Racecourse,Racetrack,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,South American Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Alderwood, Long Branch",0,0,0,0,0,0,1,1,0,0,1,1,2,0,0,1,3,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,2,1,0,0,0,0,2,0,1,0,0,1,0,1,0,0,0,2,0,0,3,0,0,1,0,0,0,0,1,0,2,0,1,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Islington Avenue, Humber Valley Village",0,0,0,4,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,1,1,0,2,0,0,0,0,0,0,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,2,0,0,0,1,0,1,0,0,0,1
3,"Kingsview Village, St. Phillips, Martin Grove ...",0,2,0,2,0,0,0,1,1,0,0,0,1,0,2,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,1,0,0,2,0,0,0,1,0,3,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Mimico NW, The Queensway West, South of Bloor,...",0,0,1,2,1,0,0,0,2,0,1,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,1,0,1,1,0,0,0,0,1,0,2


In [504]:
# Battle cell added

ref_etob_grouped = etob_grouped.drop([ 'Liquor Store',
                                      
 'Park',
 'Brewery',
 'Theater',
 'Skating Rink',
 'Movie Theater',
 'Bookstore',
 'Toy / Game Store',
 'Climbing Gym',
 'Department Store',
 'Discount Store',
 'Shopping Mall',
 'Sporting Goods Shop',
 'Pub',
 'Gym',
 'Pharmacy',
 'Tea Room',
 'Pool Hall',
 'Gastropub',
 'Indie Movie Theater',
 'Golf Course',
 'Yoga Studio',
 'American Restaurant',
 'Event Space',
 'Garden',
 'Deli / Bodega',
 'Bridge',
 'Wings Joint',
 'Scenic Lookout',
 'Bar',
 "Women's Store",
 'Furniture / Home Store',
 'Flower Shop',
 'Racetrack',
 'Hotel',
 'Beer Store',
 'Spa',
 'Train Station',
 'Nightclub',
 'Latin American Restaurant',
 'Music Venue',
 'Racecourse',
 'Hobby Shop'
 ],  axis= 1)
ref_etob_grouped.head()




,Neighbourhood,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Dessert Shop,Diner,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,General Entertainment,Gourmet Shop,Greek Restaurant,Grocery Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,South American Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,"Alderwood, Long Branch",0,0,0,1,1,1,2,0,0,3,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,1,0,0,1,0,1,2,0,0,2,1,0,0,0,0,0,0,3,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2,"Islington Avenue, Humber Valley Village",0,0,4,1,1,0,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,1,1,0,1,0,0,0,1,0,2,0,1,0
3,"Kingsview Village, St. Phillips, Martin Grove ...",2,0,2,1,0,0,1,0,2,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2,1,0,0,2,1,0,0,0,0,0,0
4,"Mimico NW, The Queensway West, South of Bloor,...",0,1,2,0,1,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,0,0,1,0,1,0,1,1,0,1,0,0,0,2,0,1,1,0,0


In [505]:
ref_etob_grouped['Total Food joints'] = ref_etob_grouped.sum(axis=1)
ref_etob_grouped

,Neighbourhood,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Dessert Shop,Diner,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,General Entertainment,Gourmet Shop,Greek Restaurant,Grocery Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,South American Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant,Total Food joints
0,"Alderwood, Long Branch",0,0,0,1,1,1,2,0,0,3,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,18
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,1,0,0,1,0,1,2,0,0,2,1,0,0,0,0,0,0,3,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,15
2,"Islington Avenue, Humber Valley Village",0,0,4,1,1,0,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,1,1,0,1,0,0,0,1,0,2,0,1,0,19
3,"Kingsview Village, St. Phillips, Martin Grove ...",2,0,2,1,0,0,1,0,2,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2,1,0,0,2,1,0,0,0,0,0,0,19
4,"Mimico NW, The Queensway West, South of Bloor,...",0,1,2,0,1,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,0,0,1,0,1,0,1,1,0,1,0,0,0,2,0,1,1,0,0,19
5,"New Toronto, Mimico South, Humber Bay Shores",0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,2,1,2,1,1,1,0,1,1,0,0,0,0,1,1,0,0,1,0,0,20
6,"Northwest, West Humber - Clairville",2,0,1,1,2,0,0,0,2,3,0,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,2,0,1,0,0,0,0,0,19
7,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0,0,1,1,1,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,1,1,0,1,0,0,0,2,0,1,1,0,0,19
8,"South Steeles, Silverstone, Humbergate, Jamest...",3,0,1,0,1,0,1,1,2,6,0,1,0,0,1,1,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,23
9,"The Kingsway, Montgomery Road, Old Mill North",0,0,2,1,2,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,1,0,0,1,0,0,0,1,1,2,0,0,0,17


In [506]:
Refined_etob_grouped = ref_etob_grouped[['Neighbourhood','Total Food joints']]
Refined_etob_grouped

,Neighbourhood,Total Food joints
0,"Alderwood, Long Branch",18
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",15
2,"Islington Avenue, Humber Valley Village",19
3,"Kingsview Village, St. Phillips, Martin Grove ...",19
4,"Mimico NW, The Queensway West, South of Bloor,...",19
5,"New Toronto, Mimico South, Humber Bay Shores",20
6,"Northwest, West Humber - Clairville",19
7,"Old Mill South, King's Mill Park, Sunnylea, Hu...",19
8,"South Steeles, Silverstone, Humbergate, Jamest...",23
9,"The Kingsway, Montgomery Road, Old Mill North",17


In [507]:
result = pd.merge(df_etob, Refined_etob_grouped, on='Neighbourhood').drop((["Borough"]), axis = 1)
result

,Postal Code,Neighbourhood,Latitude,Longitude,Total Food joints
0,M8V,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,20
1,M8W,"Alderwood, Long Branch",43.602414,-79.543484,18
2,M8X,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,17
3,M8Y,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,19
4,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,19
5,M9A,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,19
6,M9B,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,20
7,M9C,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,15
8,M9P,Westmount,43.696319,-79.532242,20
9,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,19


In [508]:
## Final file for ML algorithim

final_df = pd.merge(result, etobdmg, on='Neighbourhood') 
final_df


,Postal Code,Neighbourhood,Latitude,Longitude,Total Food joints,Density(people/km2),% change in population since 2001,Average Income
0,M8V,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,20,7588,-5.7,39186
1,M8W,"Alderwood, Long Branch",43.602414,-79.543484,18,2360,-4.0,35239
2,M8X,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,17,3403,7.9,110944
3,M8Y,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,19,3366,-1.1,51398
4,M8Z,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,19,2629,15.4,47011
5,M9A,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,19,2652,-0.1,80618
6,M9B,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,20,2249,1.0,80607
7,M9C,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,15,3507,1.0,51695
8,M9P,Westmount,43.696319,-79.532242,20,5932,5.0,35183
9,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,19,4013,-6.2,32004


In [509]:
final_clustering = final_df.drop(['Postal Code', 'Latitude', 'Longitude'], axis = 1)
final_clustering


,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income
0,"New Toronto, Mimico South, Humber Bay Shores",20,7588,-5.7,39186
1,"Alderwood, Long Branch",18,2360,-4.0,35239
2,"The Kingsway, Montgomery Road, Old Mill North",17,3403,7.9,110944
3,"Old Mill South, King's Mill Park, Sunnylea, Hu...",19,3366,-1.1,51398
4,"Mimico NW, The Queensway West, South of Bloor,...",19,2629,15.4,47011
5,"Islington Avenue, Humber Valley Village",19,2652,-0.1,80618
6,"West Deane Park, Princess Gardens, Martin Grov...",20,2249,1.0,80607
7,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",15,3507,1.0,51695
8,Westmount,20,5932,5.0,35183
9,"Kingsview Village, St. Phillips, Martin Grove ...",19,4013,-6.2,32004


In [510]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

grouped_clustering = final_clustering.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:12]

array([0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0])

In [511]:
# add clustering labels
final_clustering.insert(0, 'Cluster Labels', kmeans.labels_)


In [512]:
final_clustering

,Cluster Labels,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income
0,0,"New Toronto, Mimico South, Humber Bay Shores",20,7588,-5.7,39186
1,0,"Alderwood, Long Branch",18,2360,-4.0,35239
2,1,"The Kingsway, Montgomery Road, Old Mill North",17,3403,7.9,110944
3,0,"Old Mill South, King's Mill Park, Sunnylea, Hu...",19,3366,-1.1,51398
4,0,"Mimico NW, The Queensway West, South of Bloor,...",19,2629,15.4,47011
5,2,"Islington Avenue, Humber Valley Village",19,2652,-0.1,80618
6,2,"West Deane Park, Princess Gardens, Martin Grov...",20,2249,1.0,80607
7,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",15,3507,1.0,51695
8,0,Westmount,20,5932,5.0,35183
9,0,"Kingsview Village, St. Phillips, Martin Grove ...",19,4013,-6.2,32004


In [531]:
ft = final_clustering.merge(final_df[['Postal Code', 'Neighbourhood', 'Latitude', 'Longitude']])
ft

,Cluster Labels,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income,Postal Code,Latitude,Longitude
0,0,"New Toronto, Mimico South, Humber Bay Shores",20,7588,-5.7,39186,M8V,43.605647,-79.501321
1,0,"Alderwood, Long Branch",18,2360,-4.0,35239,M8W,43.602414,-79.543484
2,1,"The Kingsway, Montgomery Road, Old Mill North",17,3403,7.9,110944,M8X,43.653654,-79.506944
3,0,"Old Mill South, King's Mill Park, Sunnylea, Hu...",19,3366,-1.1,51398,M8Y,43.636258,-79.498509
4,0,"Mimico NW, The Queensway West, South of Bloor,...",19,2629,15.4,47011,M8Z,43.628841,-79.520999
5,2,"Islington Avenue, Humber Valley Village",19,2652,-0.1,80618,M9A,43.667856,-79.532242
6,2,"West Deane Park, Princess Gardens, Martin Grov...",20,2249,1.0,80607,M9B,43.650943,-79.554724
7,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",15,3507,1.0,51695,M9C,43.643515,-79.577201
8,0,Westmount,20,5932,5.0,35183,M9P,43.696319,-79.532242
9,0,"Kingsview Village, St. Phillips, Martin Grove ...",19,4013,-6.2,32004,M9R,43.688905,-79.554724


In [532]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        ft['Latitude'], 
        ft['Longitude'], 
        ft['Neighbourhood'], 
        ft['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [533]:
analysis = ft["Cluster Labels"].isin(["0"]) 
Cluster_0= ft[analysis]
Cluster_0

# Cluster 0 - High population density and low Average Income

,Cluster Labels,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income,Postal Code,Latitude,Longitude
0,0,"New Toronto, Mimico South, Humber Bay Shores",20,7588,-5.7,39186,M8V,43.605647,-79.501321
1,0,"Alderwood, Long Branch",18,2360,-4.0,35239,M8W,43.602414,-79.543484
3,0,"Old Mill South, King's Mill Park, Sunnylea, Hu...",19,3366,-1.1,51398,M8Y,43.636258,-79.498509
4,0,"Mimico NW, The Queensway West, South of Bloor,...",19,2629,15.4,47011,M8Z,43.628841,-79.520999
7,0,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",15,3507,1.0,51695,M9C,43.643515,-79.577201
8,0,Westmount,20,5932,5.0,35183,M9P,43.696319,-79.532242
9,0,"Kingsview Village, St. Phillips, Martin Grove ...",19,4013,-6.2,32004,M9R,43.688905,-79.554724
10,0,"South Steeles, Silverstone, Humbergate, Jamest...",23,4229,1.4,28955,M9V,43.739416,-79.588437
11,0,"Northwest, West Humber - Clairville",19,1268,-3.3,26610,M9W,43.706748,-79.594054


In [536]:
analysis = ft["Cluster Labels"].isin(["1"]) 
Cluster_1= ft[analysis]
Cluster_1

# Cluster 1  high density, with high population growht with high average income ----(Our Recommendation)

,Cluster Labels,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income,Postal Code,Latitude,Longitude
2,1,"The Kingsway, Montgomery Road, Old Mill North",17,3403,7.9,110944,M8X,43.653654,-79.506944


In [535]:
analysis = ft["Cluster Labels"].isin(["2"]) 
Cluster_2= ft[analysis]
Cluster_2

# Cluster 2  low population density, low population growth and medium average income

,Cluster Labels,Neighbourhood,Total Food joints,Density(people/km2),% change in population since 2001,Average Income,Postal Code,Latitude,Longitude
5,2,"Islington Avenue, Humber Valley Village",19,2652,-0.1,80618,M9A,43.667856,-79.532242
6,2,"West Deane Park, Princess Gardens, Martin Grov...",20,2249,1.0,80607,M9B,43.650943,-79.554724
